# Load package

In [1]:
%cd ..

E:\Coding\Python\PlasmaChemistry


In [2]:
import os
import re
import math
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
import qgrid

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout
from IPython.display import display
import ipywidgets as widgets

In [4]:
%matplotlib notebook
import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (9, 6)
plt.rcParams["font.size"] = 15

In [5]:
from plasmistry.molecule import (H2_vib_group, CO_vib_group, CO2_vib_group)
from plasmistry.molecule import (H2_vib_energy_in_eV, H2_vib_energy_in_K,
                                 CO2_vib_energy_in_eV, CO2_vib_energy_in_K,
                                 CO_vib_energy_in_eV, CO_vib_energy_in_K)
from plasmistry.io import (LT_constructor, standard_Arr_constructor,
                           chemkin_Arr_2_rcnts_constructor,
                           chemkin_Arr_3_rcnts_constructor, eval_constructor,
                           reversed_reaction_constructor, alpha_constructor,
                           F_gamma_constructor,
                           Cros_Reaction_block, Coef_Reaction_block)
from plasmistry.reactions import (CrosReactions, CoefReactions)
from plasmistry.electron import EEDF
from plasmistry.electron import get_maxwell_eedf

In [6]:
import yaml
yaml.add_constructor(u"!eval", eval_constructor)
yaml.add_constructor(u"!LT", LT_constructor)
yaml.add_constructor("!StandardArr", standard_Arr_constructor)
yaml.add_constructor("!ChemKinArr_2_rcnt", chemkin_Arr_2_rcnts_constructor)
yaml.add_constructor("!ChemKinArr_3_rcnt", chemkin_Arr_3_rcnts_constructor)
yaml.add_constructor("!rev", reversed_reaction_constructor)
yaml.add_constructor("!alpha", alpha_constructor)
yaml.add_constructor("!F_gamma", F_gamma_constructor)

# variables from widgets

`_species_list`

`_init_yaml_file_path`

`_vari_dict`

`rctn_all [dict, ...]`
 - _electron reactions_
 - _relaxation reactions_
 - _chemical reactions_
 - decom_recom reactions

`rctn_df [dataframe, ...]` 
- _species_
- _electron_
- _chemical_
- decom_recom
- _relaxation_

`rctn_instances [Reactions instances, ...]`
- _cros reactions_
- _coef reactions_

In [7]:
_species_list = [
    'E', 'H2(v0-14)', 'CO2(v0-21)', 'CO(v0-10)', 'O2', 'H2O', 'H', 'O', 'OH'
]
_init_yaml_file_path = './_yaml/test_0.yaml'
_vari_dict = dict(H2_vib_energy_in_eV=H2_vib_energy_in_eV,
                  H2_vib_energy_in_K=H2_vib_energy_in_K,
                  CO2_vib_energy_in_eV=CO2_vib_energy_in_eV,
                  CO2_vib_energy_in_K=CO2_vib_energy_in_K)
# ---------------------------------------------------------------------------- #
rctn_all = {
    'electron reactions': None,
    'relaxation reactions': None,
    'chemical reactions': None,
    'decom_recom reactions': None
}
with open(_init_yaml_file_path, 'r') as f:
    rctn_block = yaml.load(f)
rctn_all = rctn_block[-1]['The reactions considered']

rctn_df = {
    'species': None,
    'electron': None,
    'chemical': None,
    'decom_recom': None,
    'relaxation': None
}
rctn_instances = {'cros reactions': None, 'coef reactions': None}

# widgets function
`yaml file` => `rctn_all` (dict inside)
            => `rctn_df` (dataframe insdie) 
            => `rctn_instances` (instances inside)
- `get_species_from_widgets`
- `get_rctn_df_from_widgets`
- `instance_dataframe`



In [8]:
def get_species_from_widgets(_event):
    with _widgets['output']:
        print('Get species from widgets ...', end=' ')
    species = []
    for _ in _widgets['species'].value:
        if _ == 'H2(v0-14)':
            species.append('H2')
            species.extend([f'H2(v{v})' for v in range(1, 15)])
        elif _ == 'CO2(v0-21)':
            species.append('CO2')
            species.extend([f'CO2(v{v})' for v in range(1, 22)])
        elif _ == 'CO(v0-10)':
            species.append('CO')
            species.extend([f'CO(v{v})' for v in range(11)])
        else:
            species.append(_)
    rctn_df['species'] = pd.Series(species)
    assert rctn_df['species'][0] == 'E'
    with _widgets['output']:
        print('DONE!')


def get_rctn_df_from_widgets(_event):
    with _widgets['output']:
        print('Get reaction dataframe from widgets ...', end=' ')
    rctn_df['electron'] = pd.DataFrame(
        columns=['formula', 'type', 'threshold_eV', 'cross_section'])
    rctn_df['chemical'] = pd.DataFrame(
        columns=['formula', 'type', 'reactant', 'product', 'kstr'])
    rctn_df['relaxation'] = pd.DataFrame(
        columns=['formula', 'type', 'reactant', 'product', 'kstr'])
    rctn_df['decom_recom'] = pd.DataFrame(
        columns=['formula', 'type', 'reactant', 'product', 'kstr'])
    # ------------------------------------------------------------------------ #
    #    rctn_df that is generated from the widgets and rctn_all.
    # ------------------------------------------------------------------------ #
    #   electron
    for _ in _widgets['electron'].value:
        _df = rctn_all['electron reactions'][_]
        _cros_block = Cros_Reaction_block(rctn_dict=_df, vari_dict=_vari_dict)
        if _ == 'H2_ele_dis_via_b':
            factor = 1e-20
            ##TODO##
            # factor = 0.2e-21
        else:
            factor = 1
        rctn_df['electron'] = pd.concat([
            rctn_df['electron'],
            _cros_block.generate_crostn_dataframe(factor=factor)
        ],
                                        ignore_index=True,
                                        sort=False)

    # ------------------------------------------------------------------------ #
    #   relaxation
    for _ in _widgets['relaxation'].value:
        _df = rctn_all['relaxation reactions'][_]
        _coef_block = Coef_Reaction_block(rctn_dict=_df, vari_dict=_vari_dict)
        rctn_df['relaxation'] = pd.concat(
            [rctn_df['relaxation'],
             _coef_block.generate_crostn_dataframe()],
            ignore_index=True,
            sort=False)

    # ------------------------------------------------------------------------ #
    #   chemical
    for _ in _widgets['chemical'].value:
        _df = rctn_all['chemical reactions'][_]
        _coef_block = Coef_Reaction_block(rctn_dict=_df, vari_dict=_vari_dict)
        rctn_df['chemical'] = pd.concat(
            [rctn_df['chemical'],
             _coef_block.generate_crostn_dataframe()],
            ignore_index=True,
            sort=False)

    # ------------------------------------------------------------------------ #
    #    decom_recom
    for _ in _widgets['decom_recom'].value:
        _df = rctn_all['decom_recom reactions'][_]
        _coef_block = Coef_Reaction_block(rctn_dict=_df, vari_dict=_vari_dict)
        rctn_df['decom_recom'] = pd.concat(
            [rctn_df['decom_recom'],
             _coef_block.generate_crostn_dataframe()],
            ignore_index=True,
            sort=False)
    with _widgets['output']:
        print('DONE!')


# ---------------------------------------------------------------------------- #
def rctn_all_dict_keys_to_formulas(_key):
    _formula = _key.replace('_to_', ' => ')
    _formula = _formula.replace('_', ' + ')
    return _formula


# ---------------------------------------------------------------------------- #
def from_rctn_df_to_cros_instance(_df):
    split_df = _df['formula'].str.split('\s*=>\s*', expand=True)
    reactant = split_df[0]
    product = split_df[1]
    return reactant, product


def from_rctn_df_to_coef_instance(_df):
    reactant = _df['reactant']
    product = _df['product']
    kstr = _df['kstr']
    return reactant, product, kstr


# ---------------------------------------------------------------------------- #
def instance_dataframe(_):
    with _widgets['output']:
        print("Get instance from rctn_df ...", end=' ')
    # ------------------------------------------------------------------------ #
    reactant, product = from_rctn_df_to_cros_instance(rctn_df['electron'])

    rctn_instances['cros reactions'] = CrosReactions(
        species=rctn_df['species'],
        reactant=reactant,
        product=product,
        k_str=None)
    # ------------------------------------------------------------------------ #
    reactant, product, kstr = pd.Series(), pd.Series(), pd.Series()
    for _key in ['relaxation', 'chemical', 'decom_recom']:
        _r, _p, _k = from_rctn_df_to_coef_instance(rctn_df[_key])
        reactant = pd.concat([reactant, _r], ignore_index=True, sort=False)
        product = pd.concat([product, _p], ignore_index=True, sort=False)
        kstr = pd.concat([kstr, _k], ignore_index=True, sort=False)

    rctn_instances['coef reactions'] = CoefReactions(
        species=rctn_df['species'],
        reactant=reactant,
        product=product,
        k_str=kstr)
    rctn_instances['coef reactions'].compile_k_str()
    # ------------------------------------------------------------------------ #
    with _widgets['output']:
        print('DONE!')


def clean_output(_):
    _widgets['output'].clear_output()

# Set widgets and display widgets

In [9]:
# ---------------------------------------------------------------------------- #
_widgets = dict()
# ---------------------------------------------------------------------------- #
#   Set SelectMultiple widgets
# ---------------------------------------------------------------------------- #
for _, _option, _height in [
    ('species', _species_list, '400px'),
    ('electron', rctn_all['electron reactions'].keys(), '150px'),
    ('relaxation', rctn_all['relaxation reactions'].keys(), '150px'),
    ('chemical', rctn_all['chemical reactions'].keys(), '400px'),
    ('decom_recom', rctn_all['decom_recom reactions'].keys(), '400px')
]:
    if _ in ('chemical', 'decom_recom'):
        show_option = [(rctn_all_dict_keys_to_formulas(_o), _o)
                       for _o in _option]
    else:
        show_option = _option
    _widgets[_] = widgets.SelectMultiple(options=show_option,
                                         value=list(_option),
                                         layout=Layout(height=_height,
                                                       width='300px'))

# ---------------------------------------------------------------------------- #
#   Set FloatText widgets
# ---------------------------------------------------------------------------- #
for _, _value in [('electron_density', 1.0), ('Te', 1.0), ('CO2_density', 1.0),
                  ('CO2_Tvib', 1000), ('H2_density', 1.0), ('H2_Tvib', 1000),
                  ('CO_density', 1.0), ('CO_Tvib', 1000)]:
    _widgets[_] = widgets.FloatText(value=_value, layout=Layout(width='100px'))
# ---------------------------------------------------------------------------- #
#   Set Buttons
# ---------------------------------------------------------------------------- #
_button = dict()
_button['load reactions'] = widgets.Button(description='rctn_all => rctn_df')
_button['instance rctn_df'] = widgets.Button(description='=> rctn_instances')
_button['clean output'] = widgets.Button(description='clean output')

# ---------------------------------------------------------------------------- #
_widgets['output'] = widgets.Output(layout={
    'border': '2px solid blue',
    'width': '80%'
})


# ---------------------------------------------------------------------------- #
#   Widgets display
# ---------------------------------------------------------------------------- #
def _label(_str):
    return widgets.HTML(f'<b>{_str}</b>')


widgets.Text()
display(
    widgets.HBox([
        widgets.HTML('<b>Yaml File Path</b>'),
        widgets.Text(value=_init_yaml_file_path, disabled=True)
    ],
                 layout=Layout(border='solid 2px', width='80%')))
display(
    widgets.HBox(
        [
            #1
            widgets.VBox([_label('SPECIES'), _widgets['species']]),
            #2
            widgets.VBox([
                _label('ELECTRON REACTIONS'), _widgets['electron'],
                _label('RELAXATION REACTIONS'), _widgets['relaxation']
            ]),
            #3
            widgets.VBox([_label('CHEMICAL REACTIONS'), _widgets['chemical']]),
            #4
            widgets.VBox(
                [_label('DECOM_RECOM REACTIONS'), _widgets['decom_recom']])
        ],
        layout=Layout(width='80%', border='solid 2px')))
display(
    widgets.GridBox([
        widgets.HTML('<b>electron density</b>'), _widgets['electron_density'],
        widgets.HTML('<b>Te_eV</b>'), _widgets['Te'],
        widgets.HTML('<b>CO2_density</b>'), _widgets['CO2_density'],
        widgets.HTML('<b>CO2_Tvib</b>'), _widgets['CO2_Tvib'],
        widgets.HTML('<b>H2_density</b>'), _widgets['H2_density'],
        widgets.HTML('<b>H2_Tvib'), _widgets['H2_Tvib']
    ],
                    layout=widgets.Layout(
                        grid_template_columns="repeat(4, 20%)",
                        width='80%',
                        border='solid 2px')))
display(
    widgets.VBox([
        _button['load reactions'], _button['instance rctn_df'],
        _button['clean output']
    ],
                 layout=Layout(border='solid 2px',
                               display='flex',
                               align_items='stretch',
                               width='30%')))
display(_widgets['output'])
# ---------------------------------------------------------------------------- #
#   Widgets events
# ---------------------------------------------------------------------------- #
_button['load reactions'].on_click(get_rctn_df_from_widgets)
_button['load reactions'].on_click(get_species_from_widgets)
_button['instance rctn_df'].on_click(instance_dataframe)
_button['clean output'].on_click(clean_output)

GridBox(children=(HTML(value='<b>electron density</b>'), FloatText(value=1.0, layout=Layout(width='100px')), H…

Output(layout=Layout(border='2px solid blue', width='80%'))

In [15]:
rctn_df['relaxation'].to_csv('test_relaxation.dat', float_format='.2e', columns=['formula', 'kstr'], index_label='No.')

In [13]:
rctn_df.keys()

dict_keys(['species', 'electron', 'chemical', 'decom_recom', 'relaxation'])

# Prepare rctn_instances

In [11]:
eedf = EEDF(max_energy_eV=30, grid_number=300)
rctn_instances['cros reactions'].set_rate_const_matrix(
    crostn_dataframe=rctn_df['electron'],
    electron_energy_grid=eedf.energy_point)

AttributeError: 'NoneType' object has no attribute 'set_rate_const_matrix'

# Tgas function

In [ ]:
# ---------------------------------------------------------------------------- #
#   sharp down
#   slow down
# ---------------------------------------------------------------------------- #
def Tgas_func_sharp_down(t, time_end, Tgas_arc):
    if t > time_end:
        return 300
    else:
        return Tgas_arc


def Tgas_func_slow_down(t, time_end, time_cold, Tgas_arc):
    if t > time_end:
        return (Tgas_arc - 300) * math.exp(-(t - time_end)**2 / 2 /
                                           (time_cold - time_end)**2) + 300
    else:
        return Tgas_arc

# electron density function

In [ ]:
def electron_density_func(t, time_end, _density):
    if t > time_end:
        return 0
    else:
        return _density

# dndt functions

In [ ]:
_widgets['Tgas_K_0'] = widgets.Text(description='init Tgas_K:', value='3000')
_widgets['Te_eV_0'] = widgets.Text(description='init Te_eV:', value='1.5')
_widgets['ne_0'] = widgets.Text(description='init ne', value='1e20')
_widgets['time_span'] = widgets.Text(description='Time span:',
                                     value='0, 1e-1',
                                     layout=Layout(width='200px'))
_widgets['atol'] = widgets.Text(description='atol:',
                                value='1e5',
                                layout=Layout(width='200px'))
_widgets['rtol'] = widgets.Text(description='rtol:',
                                value='1e-2',
                                layout=Layout(width='200px'))
display(_widgets['Tgas_K_0'])
display(_widgets['Te_eV_0'])
display(_widgets['ne_0'])
display(_widgets['time_span'])
display(_widgets['atol'])
display(_widgets['rtol'])

In [ ]:
Tgas_0 = float(_widgets['Tgas_K_0'].value)
Te_eV = float(_widgets['Te_eV_0'].value)
ne_0 = float(_widgets['ne_0'].value)
print(f"Tgas: {Tgas_0: .0f} K \nTe_eV: {Te_eV: .1f} eV \nne_0: {ne_0: .2e} m-3")

In [ ]:
def dndt_cros(t, density_without_e, _electron_density):
    _instance = rctn_instances['cros reactions']
    _instance.set_rate_const(eedf_normalized=normalized_eedf)
    _instance.set_rate(
        density=np.hstack([_electron_density, density_without_e]))
    return _instance.get_dn()


def dndt_coef(t, density_without_e, _electron_density, Tgas_K):
    _instance = rctn_instances['coef reactions']
    _instance.set_rate_const(Tgas_K=Tgas_K)
    _instance.set_rate(
        density=np.hstack([_electron_density, density_without_e]))
    return _instance.get_dn()


def dndt_all(t, y):
    
    _e_density = electron_density_func(t, time_end, ne_0)
    
    # _Tgas_K = Tgas_func_sharp_down(t, 1e-3, Tgas_0)
    _Tgas_K = Tgas_func_slow_down(t, time_end, time_cold, Tgas_0)
    print(f"t = {t:.6e} s    Tgas_K: {_Tgas_K:.0f} K, H2: {y[0]:.1e} CO2: {y[15]:.1e}")
    dydt = dndt_cros(t, y, _e_density) + dndt_coef(t, y, _e_density, _Tgas_K)
    return dydt[1:]


def dndt_all_with_Tgas(t, y):
    pass

In [ ]:
H2_percent_seq = [
    0.99, 0.93333, 0.88889, 0.83333, 0.77778, 0.66667, 0.5, 0.33333, 0.23077,
    0.16667, 0.13043, 0.09091, 0.07407, 0.0566, 0.03, 0.01, 0
]
time_end_seq = [
    0.38564, 0.46339, 0.58923, 0.66704, 0.69614, 0.8726, 0.97919, 1.01216,
    1.05427, 1.14294, 1.06026, 1.11772, 1.13853, 1.17361, 1.26435, 1.30063,
    1.27632
]
#### ====================
# i_index = 6 for H2/CO2 = 1:1
i_index = 6
H2_percent = H2_percent_seq[i_index]
time_end = time_end_seq[i_index] * 1e-3
time_cold = time_end + 2e-2
# time_end = 1e-3

In [ ]:
# ---------------------------------------------------------------------------- #
density_0 = rctn_instances['cros reactions'].get_initial_density(
    density_dict={
        'CO2': 2.4e24 * (1 - H2_percent),
        'H2': 2.4e24 * H2_percent,
        'E': ne_0,
    })
density_without_e_0 = density_0[1:]
normalized_eedf = get_maxwell_eedf(eedf.energy_point, Te_eV=Te_eV)

# Start evolution

In [ ]:
sol = solve_ivp(
    dndt_all,
    [float(_.strip()) for _ in _widgets['time_span'].value.split(',')],
    density_without_e_0,
    method='BDF',
    atol=float(_widgets['atol'].value),
    rtol=float(_widgets['rtol'].value))
print("Solve Done!")

# Plot results

fig, ax = plt.subplots()
_Tgas = [Tgas_func_slow_down(_, 1e-3, 2e-3, Tgas_0) for _ in sol.t]
_e_density = [electron_density_func(_, 1e-3, ne_0) for _ in sol.t]
ax.semilogx(sol.t, _Tgas, marker='.')
ax_right = ax.twinx()
ax_right.plot(sol.t, _e_density, marker='.', color='red')

In [ ]:
%matplotlib notebook
_CO2_group = ['CO2'] + [f"CO2(v{_})" for _ in range(1, 22)]
_H2_group = ['H2'] + [f"H2(v{_})" for _ in range(1, 15)]
density_result = np.vstack(
    [[electron_density_func(_, time_end, ne_0) for _ in sol.t], sol.y])
density_result_df = pd.DataFrame(
    density_result, index=rctn_instances['cros reactions'].species)
densities = dict()

In [ ]:
# ---------------------------------------------------------------------------- #
#   Plot CO2 vdf
# ---------------------------------------------------------------------------- #
fig, ax = plt.subplots()
for _ in ['CO2'] + [f"CO2(v{_})" for _ in range(1,22)]:
    # for _ in ['CO2_total', 'CO', 'O2', 'O']:
    if _ == 'CO2_total':
        densities[_] = density_result_df.loc[_CO2_group, :].sum().values
    elif _ == 'H2_total':
        densities[_] = density_result_df.loc[_H2_group, :].sum().values
    else:
        densities[_] = density_result[rctn_instances['cros reactions'].species
                                      == _].transpose()
    ax.semilogx(sol.t, densities[_], marker='.', label=_)
    # ax.loglog(sol.t, densities[_], marker='.', label=_)
ax.axvline(time_end, color='gray', linestyle='-.')
ax.set_xlabel('time (s)')
ax.set_ylabel('density (m^-3)')
fig.legend(loc='upper right', fontsize='small')

## reaction info

In [ ]:
print(H2_percent)
print("H O OH")
print(
    f"{density_result_df.loc['H'].max():.1e} {density_result_df.loc['O'].max():.1e} {density_result_df.loc['OH'].max():.1e}"
)
print(f"CO output: {density_result_df.loc['CO'].values[-1]:.1e}")

In [ ]:
density_result_df.loc['CO'].values[-1]

In [ ]:
Tv_CO2 = -3380 / np.log(
    density_result_df.loc['CO2(v1)', :] / density_result_df.loc['CO2', :])
fig, ax = plt.subplots()
ax.semilogx(sol.t, Tv_CO2, marker='.')
ax.axvline(time_end, color='gray', linestyle='-.')

## show vdf

In [ ]:
# _widgets['plot_vdf'] = widgets.FloatLogSlider(value=sol.t[1], min=sol.t[1], max=sol.t[-1],step=1)
_widgets['plot_vdf'] = widgets.FloatLogSlider(value=math.log10(sol.t[1]),
                                              min=math.log10(sol.t[1]),
                                              max=math.log10(sol.t[-1]),
                                              step=0.0001,
                                              description='Time (s):')

_widgets['output_rctn'] = widgets.Output(layout={
    'border': '2px solid blue',
    'width': '80%',
    'height': '350px'
})

display(_widgets['output_rctn'])
display(_widgets['plot_vdf'])
_df_show_vdf = pd.Series(index=rctn_instances['cros reactions'].species)

_interp_result = interp1d(sol.t, density_result)


def get_CO2_vdf_at_time(_t):
    return pd.Series(
        _interp_result(_t),
        index=rctn_instances['cros reactions'].species)[_CO2_group].values


def get_H2_vdf_at_time(_t):
    return pd.Series(
        _interp_result(_t),
        index=rctn_instances['cros reactions'].species)[_H2_group].values

# ---------------------------------------------------------------------------- #
#   plot main molecule
# ---------------------------------------------------------------------------- #
fig, ax = plt.subplots()
for _ in ['CO2_total', 'H2_total', 'CO', 'H2O', 'O2', 'H', 'O', 'OH']:
    # for _ in ['CO2_total', 'CO', 'O2', 'O']:
    if _ == 'CO2_total':
        densities[_] = density_result_df.loc[_CO2_group, :].sum().values
    elif _ == 'H2_total':
        densities[_] = density_result_df.loc[_H2_group, :].sum().values
    else:
        densities[_] = density_result[rctn_instances['cros reactions'].species
                                      == _].transpose()
    ax.semilogx(sol.t, densities[_], marker='.', label=_)
    # ax.loglog(sol.t, densities[_], marker='.', label=_)
vline_time_end = ax.axvline(time_end, color='gray', linestyle='-')
vline = ax.axvline(time_end, color='black', linestyle='-.')
ax.set_xlabel('time (s)')
ax.set_ylabel('density (m^-3)')
fig.legend(loc='upper right', fontsize='small')
# ---------------------------------------------------------------------------- #
#   plot the vdf
# ---------------------------------------------------------------------------- #
fig, ax = plt.subplots()
line_0, = ax.semilogy(list(range(22)), get_CO2_vdf_at_time(0), marker='.')
# line_1, = axs[1].semilogy(list(range(15)), get_H2_vdf_at_time(0), marker='.')
# ax.set_title('CO2 density vs. v')
ax.set_xlabel('CO2 vibrational number')
ax.set_ylabel('density (m^-3)')
# ax.set_title('H2 density vs. v')
# ax.set_xlabel('H2_vdf')
# ax.set_ylabel('density')


def plot_CO2_vdf(change):
    _t = change['new']
    _ydata = get_CO2_vdf_at_time(_t)
    line_0.set_ydata(_ydata)
    ax.set_ylim(_ydata.max()*1e-30, _ydata.max())


def plot_H2_vdf(change):
    _t = change['new']
    line_1.set_ydata(get_H2_vdf_at_time(_t))

def regexp_match(_str):
    rcnt, prdt = _str.split('=>')
    if rcnt.count('CO2') > prdt.count('CO2'):
        return True
    else:
        return False
    
    
def show_top_reactions(change):
    # _formula_regexp = "(.*\s+)?O\s+.*=>.*"
    _t = change['new']
    _Tgas = Tgas_func_slow_down(_t, time_end, time_cold, Tgas_0)
    rctn_instances['coef reactions'].set_rate_const(Tgas_K=_Tgas)
    rctn_instances['coef reactions'].set_rate(density=_interp_result(_t))
    rctn_instances['cros reactions'].set_rate(density=_interp_result(_t))
    _df_coef = rctn_instances['coef reactions'].view_rate_const_and_rate()
    _df_coef = _df_coef[_df_coef['formula'].map(regexp_match)]
    # _df = _df[_df['formula'].str.match(_formula_regexp)]
    _df_cros = rctn_instances['cros reactions'].view_rate_const_and_rate()
    _df_cros = _df_cros[_df_cros['formula'].map(regexp_match)]
    _df = pd.concat([_df_cros, _df_coef], ignore_index=True)
    _df = _df.sort_values(by='rate', ascending=False)
    _widgets['output_rctn'].clear_output()
    vline.set_xdata(_t)
    with _widgets['output_rctn']:
        print(_df.head(n=20))
    
_widgets['plot_vdf'].observe(plot_CO2_vdf, names='value')
_widgets['plot_vdf'].observe(plot_H2_vdf, names='value')
_widgets['plot_vdf'].observe(show_top_reactions, names='value')

In [ ]:
rctn_instances['coef reactions'].set_rate(density=density_result[:, -1])
qgrid.show_grid(rctn_instances['coef reactions'].view_rate_const_and_rate())

In [ ]:
# ---------------------------------------------------------------------------- #
#   choose the time
# ---------------------------------------------------------------------------- #
t_chosen = 0.9e-3
# ---------------------------------------------------------------------------- #
_t_index_chosen = np.argmin(np.abs(sol.t - t_chosen))
rctn_instances['cros reactions'].set_rate(
    density=density_result[:, _t_index_chosen])
_df = rctn_instances['cros reactions'].view_rate_const_and_rate().sort_values(
    by='rate', ascending=False)


def _match(x):
    if re.match('.*H2.*=>.*H [+] H|.*CO2.*=>.*CO [+] O', x):
        return True
    else:
        return False


_df.loc[_df['formula'].map(_match)]

In [ ]:
rctn_instances['coef reactions'].set_rate_const(Tgas_K=3000,
                                                EN_Td=1.0,
                                                Te_eV=Te_eV)
rctn_instances['coef reactions'].set_rate(
    density=density_result[:, _t_index_chosen])
_df = rctn_instances['coef reactions'].view_rate_const_and_rate().sort_values(
    by='rate_const', ascending=False)


def _match(x):
    if re.match('H.*[+]\s*H\s*=>.*', x):
        return True
    else:
        return False


_df.loc[_df['formula'].map(_match)]

In [ ]:
_crostn = rctn_df['electron'].loc[282, 'formula']
_crostn

In [ ]:
_crostn = rctn_df['electron'].loc[282, 'cross_section']
_crostn

In [ ]:
_dict = _vari_dict.copy()
_dict.update({'Tgas': 3000})
_dict.update({'exp': math.exp})
eval(rctn_instances['coef reactions'].view().loc[214, 'k_str'], _dict)

In [ ]:
rctn_instances['coef reactions'].mid_variables

## Species info

In [ ]:
rctn_instances['cros reactions'].view_density(
    density_result[:, -1]).sort_values(by='density', ascending=False)

In [ ]:
print(f"H2 percent: {H2_percent}")
print(f"time end:{time_end}")
# print(f"O:  {rctn_instances['electron reactions'].view_density(sol.y[:,-1]).loc['O', 'density']:.2e}")
# print(f"H:  {rctn_instances['electron reactions'].view_density(sol.y[:,-1]).loc['H', 'density']:.2e}")
# print(f"OH: {rctn_instances['electron reactions'].view_density(sol.y[:,-1]).loc['OH', 'density']:.2e}")
##########
# max density
print('H  O  OH  CO Tvib(CO2) Tvib(H2)')
print(f"{sol.y[rctn_instances['electron reactions'].species=='H'].max(): .2e}",
      end=' ')
print(f"{sol.y[rctn_instances['electron reactions'].species=='O'].max(): .2e}",
      end=' ')
print(f"{sol.y[rctn_instances['electron reactions'].species=='OH'].max():.2e}",
      end=' ')
_df = rctn_instances['electron reactions'].view_density(sol.y[:, -1])
Tv_CO2 = -3380 / np.log(
    _df.loc['CO2(v1)', 'density'] / _df.loc['CO2', 'density'])
Tv_H2 = -5983 / np.log(_df.loc['H2(v1)', 'density'] / _df.loc['H2', 'density'])
print(f"{Tv_CO2:.0f}", end=' ')
print(f"{Tv_H2:.0f}", end=' ')

In [ ]:
sol.y[rctn_instances['electron reactions'].species == 'H'].max()

In [ ]:
# plot CO2 vdf

x = ['CO2'] + [f'CO2(v{v})' for v in range(1, 22)]
y = [_df.loc[_, 'density'] for _ in x]
plt.semilogy(range(22), y, marker='.')

In [ ]:
Tv_CO2 = -3380 / np.log(
    _df.loc['CO2(v1)', 'density'] / _df.loc['CO2', 'density'])
Tv_H2 = -5983 / np.log(_df.loc['H2(v1)', 'density'] / _df.loc['H2', 'density'])
print(f"Tvib(CO2): {Tv_CO2:.0f} K")
print(f"Tvib(H2): {Tv_H2:.0f} K")